<p style="text-align: center;font-size: 40pt">Use case 1</p>

\section{Search and Rescue} \label{sec:appSearchRescue}
%=========================================================

% Describe the application
Within the framework of the European project NIFTi (FP7-ICT-247870), novel solutions were assembled together and tested in order to support firemen and first responders in Search \& Rescue missions.
A first use of 3D maps is to help plan the of strategic deployment of responders in environment were humans can only intervene for a limited time.
Those situations include nuclear incident, chemical spill, unstable supporting structures, and excessive heat conditions.
When tele-operated, 3D maps can be used to provide the user with situation awareness and support critical decisions about risky platform motions.
This type of application often has a limited communication range, leading to an increased need for autonomous behavior such as navigation~\citep{2013_Colas_IROS_3D}.
More autonomy also means more onboard computation in case of communication breakdown.
In such situations, onboard localization is essential to return the platform to a location were wireless communication can be reestablished.

Apart from increasing pressure on real-time solutions, Search \& Rescue environments cover a large spectrum of possibilities.
For example, deployments can happen in a well-structured nuclear plant, in a partially-collapsed building or outdoors in a case of a train chemical spill.
In an advanced robot-human collaborative intervention, dynamic elements created by the other agents (e.g., firemen, other robots, etc.) acting on the field need to be considered in the registration solution.
Moreover, dynamic elements at a global level (e.g., building collapsing during the exploration) can happen.
In the case of relatively contained situations, where humans have difficulty accessing a restricted intervention area, very little dynamic elements are expected (e.g., the Fukishima Daiichi nuclear disaster in 2011).
In the cases presented in this section, applications were demonstrated with few dynamic elements and without the need to identify them.

% Describe the platform
The platform deployed in the field is called NiftiBot (\autoref{fig:robotNifti}).
Its mobility is provided by two tracked bogies linked by a differential to facilitate the crossing of uneven terrain.
Moreover, two flippers per track allow an active control of the platform orientation and offer an extended range for gap traversals.
The mechanical configuration of the robot enables it to climb slopes up to 45$^\circ$, including stairs.
The robot fits in a bounding box of 0.17\,m$^3$ volume and weights approximately 20\,kg.
The primary sensor used for registration was a 2D rotating SICK LMS-151 laser, with its rotation axis pointing toward the front of the robot.
The aggregation of 2D scans used the motion information of the platform to reconstruct 3D scan at 0.35\,Hz.
A typical 3D scan contained 55,000 points.
Two other sensors can be found on the platform, namely a \acx{gps}-aided \acx{imu} (X-sens MTI-G ) and an omnidirectional camera (PointGrey Ladybug2).
The velocity of the platform can be considered slow (0.3\,m/s), especially during teleoperation where delicate motion are required.
The pre-alignment information used as input for the registration was based on a Kalman filter fusing the \acx{imu} and the odometry information.
The large error on motion estimates came from the vibration of the tracks, the large contact surface of the tracks on the ground, and the fact that the platform often collides with obstacles.
Therefore, smooth and continuous motion models can easily break, thus simple prediction models (e.g., constant velocity) are not applicable.


\begin{figure}[htb]
\centering
	\includegraphics[width=0.99\textwidth]{robot_nifti}
	\caption{Photograph of NiftiBot, the main platform used for Search \& Rescue demonstrations in an unstructured environment. 
The 3D sensor, Sick LMS-151, is positioned in front of the robot, in-between the two white tracks. 
The extra motorized rotation axis is pointing in the forward direction of the robot.}
	\label{fig:robotNifti}
\end{figure}

% Describe the solution
The configuration of the rotating laser produces a high density of points in front of the robot, which is desirable to predict collision, but not beneficial to the registration minimization. 
Thus, we forced the maximal density to 100 points per m$^3$ after having randomly subsampled the point cloud in order to finish the registration and the map maintenance within 2 seconds.
We expected the error on pre-alignment of the 3D scans to be less than 0.5\,m based on the velocity of the platform and the number of time \icp would be computed per second. 
We used this value to limit the matching distance.
We also removed paired points with an angle difference larger than 50$^\circ$ to avoid, for example, points from different side of a wall to match together.
This happen often when the robot is moving through different rooms.
As for the global map, we maintained a density of 100 points per m$^3$ every time a new input scan was merged in it.
A maximum of 600,000 points were kept in memory to avoid degradation of the computation time performance when exploring a larger environment than expected.
The complete list of modules used with their main parameters can be found in \autoref{tab:icpConfigAppNifti}.


\begin{table}[htp]
\caption{Configuration of the \icp chain for the Search \& Rescue mapping applications. 
The definition of the column \emph{Step} follows \autoref{chap:icpSolutions}. 
The names used in the column \emph{Module} refer to specific implementation documented in the open source library \texttt{libpointmatcher}.
}
\label{tab:icpConfigAppNifti}
\centering
\footnotesize
\begin{tabularx}{\textwidth}{l l X}

\toprule
\emph{Step} & \emph{Module} & \emph{Description} \\
\midrule
DF\textsubscript{read}
  & \texttt{SimpleSensorNoise} & Add uncertainty for the SICK LMS sensor as observed in \citep{Pomerleau:2012wd}. \\
  & \texttt{SamplingSurfaceNormal} & Keep 80\,\% of the points, while extracting surface normals based on 20 \acx{nn}. \\
  & \texttt{ObservationDirection} & Add vector pointing toward the origin of the sensor.\\
  & \texttt{OrientNormals} & Orient surface normals toward the observation direction.\\
  & \texttt{MaxDensity} & Subsample to keep point with density of 100\,pts/m$^3$.\\
  
\midrule
DF\textsubscript{ref}
  & \texttt{SurfaceNormal} & Compute normal and density with  20 \acx{nn} and an approximation factor $\epsilon=3.16$. \\
  & \texttt{MaxDensity} & Subsample to keep point with density of 100\,pts/m$^3$.\\
  & \texttt{MaxPointCount} & Subsample 70\,\% if there is more than 600,000 points.\\		
  
\midrule
MF & \texttt{KDTree} & Use an approximate kD-tree with a maximum matching distance of 0.5\,m and an approximation factor of $\epsilon=3.16$.\\
\midrule
OF 	
  & \texttt{TrimmedDist} & Keep 80\,\% closest paired points. \\
  & \texttt{SurfaceNormal} & Remove paired points with normals angle larger than 50$^\circ$. \\
  
\midrule
EM 		
  & \texttt{PointToPlane} & Objective function using point-to-plane \mbox{error}.\\
  
\midrule
TC 
  & \texttt{Differential} & Stop after a minimum error below 0.01\,m and 0.001\,rad. \\
  & \texttt{Counter} & Stop after the iteration count reached 40. \\
  & \texttt{Bound} & Stop if transformation increases beyond 5.0\,m and 0.8\,rad. \\
\bottomrule

\end{tabularx}
\raggedright
\scriptsize
\emph{Legend}:\\ DF\textsubscript{read} = Data Filters for readings, DF\textsubscript{ref} = Data Filters for references, MF = Matching Function, OF = Outlier Filters, EM = Error Minimizer, TC = Transformation Checker.

\end{table}

%--------------------------------------------------
\subsection{Indoor Preliminary Tests}

To test the range of the platform motions and to demonstrate the need for 3D reconstructions, we ran an experiment in our laboratory leading to the reconstruction of a full staircase (\autoref{fig:7floorMap}).
The robot started its path in an office located on the E-floor, and was driven down a staircase two floors below (C-floor, in the basement).
The robot was controlled using a joystick by an operator following it throughout the path.
The robot was then driven six floors up to the I-floor using the same staircase.
In this application, the robot acquired scans from a stop-and-go strategy with a scan taken roughly every 2\,m.


\begin{figure}[htb]
\centering
	\includegraphics[width=0.99\textwidth]{7_floors}
\caption{Mapping of a 7-floor staircase using a Search \& Rescue robot.
\emph{Right}: Side view of the resulting map with the floors colored based on elevation.
\emph{Left}: Top view of the E-floor with the ceiling removed and the points colored based on elevation, red being higher, blue lower.
}
\label{fig:7floorMap}
\end{figure}

The complete map was processed onboard the robot.
Because the information of the past environment was fused in the global map while the robot went down the stairs, the drift in the localization was considerably reduced on the way up.
This experiment comprised two critical moments: (1) when the robot moved out of the office and (2) when the robot entered the basement (C-floor).
In both situations the overlap between the new scan and the global map was minimal.
This information was known by the operator, so more scans were taken at those moments to avoid large deviations in the global map.

%--------------------------------------------------
\subsection{Rail Yard}

On two occasions, the NIFTi platform was tested outdoors in a rail yard, with the permission of the \textit{Depo kolejov\'{y}ch vozidel Praha} (Prague Depot of Rail Vehicles, Czech Republic).
In the first experiment, the robot was also driven in the yard by operators who were following the platform.
The 3D reconstruction is shown in \autoref{fig:nifti_railyard_1}.
The robot started its journey at one corner of a railcar, going along the railcar flank to the other corner and then, turned back to the starting position passing through the vegetation located on the other side of the railcar.
Even if the path contained a loop, the precision of the registration was accurate enough to properly match the first scan with the ones recorded at the end.


\begin{figure}[htb]
\centering
\begin{tabular}{c}
	\includegraphics[width=0.99\textwidth]{nifti_railyard_1_side}\\
	\includegraphics[width=0.99\textwidth]{nifti_railyard_1_top}
\end{tabular}
\caption{Deployment of the NiftiBot in a rail yard with a single railcar and dense vegetation.
\emph{Top}: Side view of the reconstructed environment with the railcar in the middle and the vegetation behind.
\emph{Bottom}: Top view of the reconstructed environment.
For both images, colors of the point clouds were chosen to ease the comprehension of the 3D-scene.
}
\label{fig:nifti_railyard_1}
\end{figure}

In the second experiment, the robot was driven to explore inside an old railcar where a person was standing still in the shadow to test in parallel the capability of the thermal camera.
The robot then went out of the railcar, crossed dense vegetation, followed the side of a more modern railcar and stopped in front of it, where a second operator was captured in the global map.

\begin{figure}[htb]
\centering
\begin{tabular}{c}
	\includegraphics[width=0.90\textwidth]{nifti_railyard_2_side_text}\\
\end{tabular}
\caption{Bird's-eye view of the second experiment where the robot explored inside an old wooden railcar, crossed some dense vegetation and finished its path around a more modern railcar.
}
\label{fig:nifti_railyard_2}
\end{figure}

For both experiments, the robot acquired scans with a stop-and-go strategy.
The scans were gathered at uneven distances (up to 8\,m apart) by operators without prior knowledge about critical situations.
All the 3D scans were processed offline four times faster than the speed at which they were recorded.
Those experiments are a good example of semi-structured environments, where trees and dense vegetation share the scene with planar surfaces from trains.
%We thank people from the Czech Technical University (Prague), who were partners in the NIFTi consortium, for recording and generously providing the data.

%--------------------------------------------------
\subsection{Collapsed Church}

In May 2012 a sequence of earthquakes hit the Emilia-Romagna region, Northern Italy, with a magnitude of 5.8.
Three month later, NIFTi partners deployed the platform with the support of the \textit{Vigili del Fuoco} (National Fire-watchers' Corps of Italy) and the \textit{Beni Culturali} (Ministry of Culture of Italy) in Mirandola for a damage assessment mission. 
One of the visited sites was the \textit{Chiesa di San Francesco d'Assisi}, in which 3D scans were recorded.
The robot started outside the church, crossed a door and realized a straight line, navigating on the cluttered floor of the western gallery of the church (\autoref{fig:nifti_church}).
One can observe on the reconstruction the pillars and arches supporting the remaining roof of the church.
The level of damage of the church was quite important, thus limiting the exploration possibilities of the platform, as depicted in \autoref{fig:nifti_church_photo}.

\begin{figure}[htb]
%\begin{SCfigure}[.35][htbp]
%\begin{wide}
\centering
\begin{tabular}{c}
	\includegraphics[width=0.99\textwidth]{nifti_san_francesco_side}\\
	\includegraphics[width=0.99\textwidth]{nifti_san_francesco_top}
\end{tabular}
\caption{Reconstruction of the \textit{Chiesa di San Francesco d'Assisi} with the color following elevation.
\emph{Top}: Side view of the reconstruction.
\emph{Bottom}: Top view of the church.
}
\label{fig:nifti_church}
%\end{wide}
%\end{SCfigure}
\end{figure}


\begin{figure}[htb]
%\begin{adjustwidth*}{-\oddsidemargin}{-\evensidemargin}
\centering
	\includegraphics[width=0.49\textwidth]{nifti_san_francesco_photo}
	\includegraphics[width=0.49\textwidth]{nifti_san_francesco_inside}
\caption{Comparison of the point cloud reconstruction with a photograph taken during exploration.
\emph{Left}: Photograph of the western gallery with the collapse roof on the right.
\emph{Right}: Front view of the reconstruction.
}
\label{fig:nifti_church_photo}
%\end{adjustwidth*}
\end{figure}


The platform was remotely operated from a control station situated outside the church and was able to continuously scan the environment while moving in the environment.
Again, all the 3D scans were processed offline four times faster than the speed at which they were recorded.
It is another example of semi-structured environment but, in this case, the unstructured part comes from rubble following the partial collapse of the church. 
%We thank our NIFTi collaborators from University La Sapienza (Rome) for recording and generously providing the data.

%--------------------------------------------------
\subsection{Collaborative Mapping} \label{sec:collabMapping}

Within the framework of the European project \emph{sFly} (FP7-ICT-231855), three micro-helicopters (AscTec Firefly) were deployed over a Search \& Rescue training site in Zurich, Switzerland.
The platform used is shown in \autoref{fig:robot_sfly} flying over a collapsed concrete building.
The three Fireflies were sent so that each one covered a pre-determined part of the environment and streamed back images to a control station.
The collected images were used by the ETH Computer Vision and Geometry Group to reconstruct a 3D representation of the environment explored (\autoref{fig:resultsZurich} - \emph{Bottom left}).

\begin{figure}[htbp]
\centering
	\includegraphics[width=0.99\textwidth]{robot_sfly}
\caption{Photograph of one of the three AscTec Fireflies used to map the environment in collaboration with the NiftiBot. 
}
\label{fig:robot_sfly}
\end{figure}

Another map was generated using a ground platform (i.e., NiftiBot).
The robot was tele-operated on a road around the main collapsed building presented in \autoref{fig:resultsZurich}, for a path totaling 110\,m long.
The operator had a good prior knowledge of the environment before driving the robot around from a control station.
The large road coupled with the awareness of the environment contributed to increase the velocity of the robot while exploring the area.
The resulting map of the ground robot and the map of the Fireflies were then fused using a standalone \icp implementation taken from our registration library \texttt{libpointmatcher}.
Both map were having roughly 300,000 points and were registered using the configuration of \autoref{tab:icpConfigAppcallaborative}.
The final map is depicted at the bottom right of \autoref{fig:resultsZurich}.


\begin{table}[htp]
\caption{Configuration of the \icp chain for the collaborative mapping applications.
The definition of the column \emph{Step} follows \autoref{chap:icpSolutions}. 
The names used in the column \emph{Module} refer to specific implementation documented in the open source library \texttt{libpointmatcher}.
}
\label{tab:icpConfigAppcallaborative}
\centering
\footnotesize
\begin{tabularx}{\textwidth}{ll X}

\toprule
\emph{Step} & \emph{Module} & \emph{Description} \\
\midrule
DF\textsubscript{read} 	
  & \texttt{RandomSampling} & Keep randomly 15\,\% of the points. \\
\midrule
DF\textsubscript{ref} 	
  & \texttt{SamplingSurfaceNormal} & Keep 15\,\% of the points, while extracting surface normals based on 50 \acx{nn}. \\
\midrule
MF 
  & \texttt{KDTree} & Use a kD-tree with a maximum matching distance of 1.0\,m and match a point from the reading to 5 others in the reference.\\
\midrule
OF 	
  & \texttt{TrimmedDist} & Keep 80\,\% closest points. \\
\midrule  
EM
  & \texttt{PointToPlane} & Objective function using point-to-plane \mbox{error}. \\
\midrule
TC 
  & \texttt{Differential} & Stop after a minimum error below 0.01\,m and 0.001\,rad. \\
  & \texttt{Counter} & Stop after the iteration count reached 100 \\
\bottomrule

\end{tabularx}
\raggedright
\scriptsize
\emph{Legend}:\\ DF\textsubscript{read} = Data Filters for readings, DF\textsubscript{ref} = Data Filters for references, MF = Matching Function, OF = Outlier Filters, EM = Error Minimizer, TC = Transformation Checker.
\end{table}


\begin{figure}[htb]
\centering
\begin{tabular}{@{}c@{ }c@{}}
	\includegraphics[width=0.47\textwidth]{zurich_trainingSite} &
	\includegraphics[width=0.47\textwidth]{zurich_nifti_new}
	\\
	\includegraphics[width=0.47\textwidth]{zurich_vision_new_text} &
	\includegraphics[width=0.47\textwidth]{zurich_combined_new}
\end{tabular}
\caption[Resulting maps of the Zurich firefighter training site.]
{Resulting maps of the Zurich firefighter training site.
\emph{Top right}: Photograph of the training site with a partially collapsed tower in the middle.
\emph{Top left}: Top view of the reconstruction realized with the data from the ground robot with the same tower in the middle of the map.
\emph{Bottom left}: Top view of the reconstruction realized with the data from the three Fireflies.
\emph{Bottom right}: Top view of the combined map.
Note that the color correspond the elevation: light gray is low, dark blue is high.
}
\label{fig:resultsZurich}
\end{figure}

At that time, the Kalman filter used to fused the odometry with the \acs{imu} was not well tuned and bias in the estimation induced drift on yaw estimates.
Roughly, a constant drift of 5$^\circ$/s was estimated visually.
The scans were gathered while the ground platform was moving, which generated a larger localization error than prior experiments.
A total of four runs were recorded with the ground platform: (1) continuous scanning, turning clockwise around the main building; (2) continuous scanning, counterclockwise; (3) stop-and-go scanning, clockwise; and (4) stop-and-go scanning, counterclockwise.
Two experiments out of four closed the loop with a negligible error at the closing point.
Surprisingly, the successful runs were the ones turning counterclockwise, contradicting our first intuition that stop-and-go scanning would be more accurate.
This experiment highlighted the importance of correcting \acx{imu} drift with an external registration algorithm and also showed that the robot could scan while moving.
We selected the resulting representation of the second run to fuse both sources of information (i.e., laser and camera) in a common 3D reconstruction.


%--------------------------------------------------
\subsection{Artor - Autonomous Rough Terrain Outdoor Robot}
\label{sec:app_Artor}

% Describe the application
Another platform was developed in parallel to the NiftiBot by the \acx{asl}.
Modifications were done over a LandShark from Black-I Robotics, in collaboration with RUAG and Armasuisse.
The aim of the project was to develop techniques for reliable autonomous navigation of a wheeled robot in rough, outdoor terrain.
% Describe the platform
The robot, named Artor, was much larger than NiftiBot, with a volume of 0.96\,m$^3$ and an approximate weight of 350\,kg (\autoref{fig:robot_arthor}).
Three wheels on each side of the robot gave the same traction as tracked vehicles, while simplifying the maintenance of the locomotion system.
Odometry suffered from the same large rotation error problem as NiftiBot because of the unknown friction between the ground and the wheels.
The robot can drive at a maximum speed of 4.5\,m/s but was usually driven at around 1.2\,m/s during the presented experiments.
The motion of the platform can be smooth on the pavement but in off-road situation, the motion can be more rough, and at high speed the orientation can change quickly.
Odometry computation can lead to large error that is mainly caused by the high friction of the wheels on the ground.
Given the early development stage of the platform, only the wheel odometry was available as prior information for the registration module.
% Are you sure about 5Hz? if _we_ thottle, we should be explicit about it
% FP: I confirmed with the row bag and with Krusi, he was not recording the 15 Hz.
The main sensor used in this experiment is the Velodyne HDL-32E, which produces around 50,000 points per 3D scan (i.e., not all beams always returned a range value) and were recorded at 5\,Hz (i.e, half of the sensor nominal frequency).
Other sensors present on the platform were: two SICK LMS-151 (front and rear), high-resolution zoom camera, thermal camera and a \acx{gps}-aided \acx{imu}.

\begin{figure}[htb]
\centering
	\includegraphics[width=0.90\textwidth]{robot_artor}
\caption{Photograph of Artor, a Search \& Rescue robot specialized for outdoor applications.
}
\label{fig:robot_arthor}
\end{figure}

% Describe the solution
The critical element for real-time processing is the amount of points that needs to pass at high rate through the registration module.
The tuning evolved from the parameters and filters selected for NiftiBot with the aim of increasing the registration speed for Artor.
We first randomly removed 85\,\% of the points to ensure a stronger data reduction.
We also kept a lower density of points because the platform usually covered larger areas than the NifitBot during a typical deployment.
The complete list of modules used with their main parameters are listed in \autoref{tab:icpConfigAppArtor}.
% Describe 3D results
The Artor robot was driven over different types of terrain in a specialized testing facility in Wachtberg, Germany.
The path of the platform was 340\,m long, following a rectangular shape and no noticeable errors were found at the loop closure.
The robot realized the same loop for a second time using the global representation of the first loop without any problems.
An overlay of the 3D map top view with an orthogonal projection of an aerial image is provided in \autoref{fig:arthor_map} - \emph{Top right}.

\begin{figure}[hbt]
\centering
	\includegraphics[width=0.44\textwidth]{arthor_wachtberg_overlaid}
	\includegraphics[width=0.45\textwidth]{arthor_map}
	\\
	\vspace{3pt}
	\includegraphics[width=0.90\textwidth]{arthor_map_side}
\caption[Reconstruction of the testing facilities.]
{Reconstruction of the testing facilities.
\emph{Top right}: Overlay of the 3D map with an aerial view. 
\emph{Top left}: Top view of the reconstruction.
\emph{Bottom}: Bird's-eye view of the reconstruction.
The color is based on the elevation of the points, light gray being low, dark blue being high.
The aerial image was provided by \emph{Bundesamt f\"{u}r Kartographie und Geod\"{a}sie}, Frankfurt, Germany - \url{http://www.bkg.bund.de}.
}
\label{fig:arthor_map}
\end{figure}

The data were processed offline at the same rate as the recorded one.
The operator was driving the platform around to test the mobility capability of the robot, without explicitly considering any registration limitations.
On an open terrain, the solution proposed can manage a global and dense representation of the environment, even with the high turn rate of the robot.
This might be more challenging for a solution based on visual odometry to avoid drift in those conditions.


\begin{table}[htp]
\caption{Configuration of the \icp chain for the Artor mapping applications.
The definition of the column \emph{Step} follows \autoref{chap:icpSolutions}. 
The names used in the column \emph{Module} refer to specific implementation documented in the open source library \texttt{libpointmatcher}.
}
\label{tab:icpConfigAppArtor}
\centering
\footnotesize
\begin{tabularx}{\textwidth}{ll X}

\toprule
\emph{Step} & \emph{Module} & \emph{Description} \\
\midrule
DF\textsubscript{read}
  & \texttt{RandomSampling} & Keep randomly 5\,\% of the points. \\
  & \texttt{SurfaceNormal} & Compute normal with  20 \acx{nn} and an approximation factor $\epsilon=3.16$. \\
  & \texttt{ObservationDirection} & Add vector pointing toward the origin of the sensor.\\
  & \texttt{OrientNormals} & Orient surface normals toward the observation direction.\\
  & \texttt{MaxDensity} & Subsample to keep point with density of 50\,pts/m$^3$.\\
\midrule
DF\textsubscript{ref}
  & \texttt{MaxDist} & Keep points within a radius of 70\,m from the last sensor pose.\\
  & \texttt{SurfaceNormal} & Compute normal and density with  20 \acx{nn} and an approximation factor $\epsilon=3.16$. \\
  & \texttt{MaxDensity} & Subsample to keep point with density of 10\,pts/m$^3$.\\
  & \texttt{MaxPointCount} & Subsample 70\,\% if there is more than 600,000 points.\\

\midrule
MF 
  & \texttt{KDTree} & Use an approximate kD-tree with a maximum matching distance of 0.5\,m and an approximation factor of $\epsilon=3.16$.  \\
\midrule
OF
  & \texttt{TrimmedDist} & Keep 80\,\% closest paired points. \\
  & \texttt{SurfaceNormal} & Remove paired points with normals angle larger than 90$^\circ$. \\
\midrule
EM
  & \texttt{PointToPlane} & Objective function using point-to-plane \mbox{error}.\\
\midrule
TC 
  & \texttt{Differential} & Stop after a minimum error below 0.01\,m and 0.001\,rad.\\
  & \texttt{Counter} & Stop after the iteration count reached 40. \\
  & \texttt{Bound} & Stop if transformation increases beyond 5.0\,m and 0.8\,rad.\\
\bottomrule
\end{tabularx}
\raggedright
\scriptsize
\emph{Legend}:\\ DF\textsubscript{read} = Data Filters for readings, DF\textsubscript{ref} = Data Filters for references, MF = Matching Function, OF = Outlier Filters, EM = Error Minimizer, TC = Transformation Checker.
\end{table}